In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
from dataset import get_labelled, get_unlabelled, _preload
from Commit import CommitFactory
from Model import CommitDiffModelFactory

In [2]:
CONTEXT_SIZE = 12
BAG_SIZE = 256
ENCODER = 0
OUTPUT_SIZE = 256

In [3]:
Commit = CommitFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)
CommitDiffModel = CommitDiffModelFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, OUTPUT_SIZE=OUTPUT_SIZE)

In [4]:
_preload()

Loading Commit lookup table
Loading file ../data/commit_lookups/supervised_commit_data_lookup0-1000.pickle
Appending pickle of length: 184 , new dict length: 184
Loading file ../data/commit_lookups/commit_data_lookup12500-15000.pickle
Appending pickle of length: 2497 , new dict length: 2681
Loading file ../data/commit_lookups/commit_data_lookup10000-12500.pickle
Appending pickle of length: 2496 , new dict length: 5177


In [5]:
X_train, X_test, y_train, y_test = get_labelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating X_train:   0%|          | 0/97 [00:00<?, ?it/s]

Generating X_train:   0%|          | 0/97 [00:00<?, ?it/s]

Generating X_train:   0%|          | 0/70 [00:00<?, ?it/s]

Generating X_train:   0%|          | 0/70 [00:00<?, ?it/s]

In [6]:
X_train_unsupervised = get_unlabelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

  0%|          | 0/5177 [00:00<?, ?it/s]

In [7]:
print("Train set size", len(y_train))
print("Train set split", np.sum(y_train)/len(y_train))
print("Test set size", len(y_test))
print("Test set split", np.sum(y_test)/len(y_test))
print("Unsupervised Train Size", len(X_train_unsupervised))

Train set size 132
Train set split 0.5833333333333334
Test set size 33
Test set split 0.5454545454545454
Unsupervised Train Size 5177


In [8]:
model = CommitDiffModel()
model.initialize(ENCODER)

In [9]:
model.fit_siam(np.array(X_train_unsupervised), epochs=8, batch_size=64, verbose=1)

Epoch 1/8
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
81/81 [==============================] - 7s 32ms/step - loss: -0.5725
Epoch 2/8
81/81 [==============================] - 2s 28ms/step - loss: -0.6486
Epoch 3/8
81/81 [==============================] - 2s 28ms/step - loss: -0.6650
Epoch 4/8
81/81 [==============================] - 2s 30ms/step - loss: -0.6838
Epoch 5/8
81/81 [==============================] - 3s 34ms/step - loss: -0.6941
Epoch 6/8
81/81 [==============================] - 3s 34ms/step - loss: -0.6769
Epoch 7/8
81/81 [==============================] - 3s 31ms/step - loss: -0.6930
Epoch 8/8
81/81 [==============================] - 3s 35ms/step - loss: -0.6999


In [10]:
model.fit_binary_classification(X_train, np.array(y_train), epochs=8, batch_size=64, verbose=1)

Epoch 1/8
3/3 [==============================] - 4s 21ms/step - loss: 15871484928.0000
Epoch 2/8
3/3 [==============================] - 0s 22ms/step - loss: 0.5833
Epoch 3/8
3/3 [==============================] - 0s 19ms/step - loss: 0.5833
Epoch 4/8
3/3 [==============================] - 0s 19ms/step - loss: 0.5833
Epoch 5/8
3/3 [==============================] - 0s 18ms/step - loss: 0.5833
Epoch 6/8
3/3 [==============================] - 0s 17ms/step - loss: 0.5833
Epoch 7/8
3/3 [==============================] - 0s 18ms/step - loss: 0.5833
Epoch 8/8
3/3 [==============================] - 0s 17ms/step - loss: 0.5833


In [11]:
model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)

0.5454545617103577